In [2]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install faiss-cpu
!pip install langchain_experimental
!pip install "langchain[docarray]"

  Using cached langchain_experimental-0.3.4-py3-none-any.whl.metadata (1.7 kB)
Using cached langchain_experimental-0.3.4-py3-none-any.whl (209 kB)


In [7]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken

In [ ]:
import os

# Prompt the user for their OpenAI API key
file = open("OPENAI_API_KEY.txt")

api_key = file.read()

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = api_key

# Optionally, check that the environment variable was set correctly
print("OPENAI_API_KEY has been set!")


OPENAI_API_KEY has been set!


In [5]:
from langchain.text_splitter import CharacterTextSplitter

In [10]:
txt_file_path = 'context.txt'
loader = TextLoader(file_path=txt_file_path, encoding="utf-8")
data = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
data = text_splitter.split_documents(data)

In [11]:
# Create vector store
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(data, embedding=embeddings)

In [12]:
# Create conversation chain
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4o")
memory = ConversationBufferMemory(
memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        memory=memory
        )

C:\Users\bqtha\AppData\Local\Temp\ipykernel_27676\360721615.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.7, model_name="gpt-4o")
C:\Users\bqtha\AppData\Local\Temp\ipykernel_27676\360721615.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [13]:
query = "When should we plant corn?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

C:\Users\bqtha\AppData\Local\Temp\ipykernel_27676\819008554.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = conversation_chain({"question": query})


"The best time to plant corn typically depends on your location and climate. Generally, corn is planted in the spring after the last frost date, when the soil temperature reaches at least 50°F (10°C). In many regions, this means planting corn from late April to early June. It's important to ensure that the soil is warm enough, as planting in cold soil can lead to poor germination and growth. Always consider local agricultural guidelines for the most accurate planting times in your area."